In [1]:
import pandas as pd
import numpy as np
from pathlib import Path

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

In [3]:
data_dir = Path('../data/processed')
assert data_dir.exists()

In [4]:
onehz = pd.read_parquet(data_dir / 'cups_1hz.parquet').set_index('timestamp').asfreq('1s') # fill mising timestamps with nan

In [5]:
onehz.head(3)

,Air_Temp_87m,DeltaT_122_87m,Dewpt_Temp_122m,Dewpt_Temp_87m,PRECIP_INTEN,Cup_WS_C1_130m,Cup_WS_122m,Cup_WS_C1_105m,Vane_WD_122m,Vane_WD_87m
timestamp,,,,,,,,,,
2018-12-31 17:00:00,-15.698145,-0.286780,-14.127690,-14.877580,2.939480,6.966611,7.332340,6.931492,22.893036,28.280596
2018-12-31 17:00:01,-15.555123,-0.286780,-14.108216,-14.851624,2.944349,7.140418,7.182121,6.859962,28.400145,23.877924
2018-12-31 17:00:02,-15.672141,-0.290412,-14.095233,-14.910026,2.911893,7.181488,6.682454,6.982190,27.367561,21.780317


In [6]:
# Don't need to know the root cause of a flag anymore, so
# collapse separate labels into a single binary flag with .any(axis=1)
labels = {f.stem.rsplit('.', maxsplit=1)[0] : pd.read_parquet(f).any(axis=1)
          for f in (data_dir / 'labels').glob('*.parquet.gz')
         }

In [7]:
labels['Cup_WS_C1_105m'].head(3)

timestamp
2018-12-31 17:00:00    False
2018-12-31 17:00:01    False
2018-12-31 17:00:02    False
dtype: bool

In [8]:
# apply labels to data
for col, label in labels.items():
    onehz.loc[:, col].where(~label, inplace=True) # set NaN

In [9]:
# Due to inherited processing, set all 0 wind speed values to NaN (interpolate later)
onehz.loc[:, ['Cup_WS_C1_105m', 'Cup_WS_C1_130m']] = onehz.loc[:, ['Cup_WS_C1_105m', 'Cup_WS_C1_130m']].replace(0, np.nan)

In [10]:
onehz.count()

Air_Temp_87m       30654998
DeltaT_122_87m     30654998
Dewpt_Temp_122m    30654998
Dewpt_Temp_87m     30654998
PRECIP_INTEN       30654998
Cup_WS_C1_130m     29414795
Cup_WS_122m        30654994
Cup_WS_C1_105m     29415189
Vane_WD_122m       30654998
Vane_WD_87m        30654998
dtype: int64

# Aggregates
Reproduce the aggregates available to currently-deployed meteorological masts. These (and derived combinations of them) will be the input features of the models.

* mean
* variance
* min
* max
* "3-second gust" = max(rolling 3-second mean). For anemometers only.
* count

In [11]:
anemometers = ['Cup_WS_C1_130m', 'Cup_WS_C1_105m']
# Exclude Cup_WS_122m because it is a lower quality sensor in a poor mounting position
support = ['Air_Temp_87m', 'DeltaT_122_87m']
vanes = ['Vane_WD_122m', 'Vane_WD_87m']
# Exclude dew point and precipitation because most commercial systems don't have those sensors

In [12]:
del labels, label # memory for laptop runs

In [13]:
aggs = onehz[anemometers + support].resample('10min').agg(['mean', 'var', 'min', 'max', 'count'])

In [56]:
# wind vanes need custom calculations due to cyclic angle measurements (wrap around 359 -> 0)
from src.data.aggregate import wind_dir_from_vec, mean_vec_from_dirs
from src.features.build_features import angular_difference

def vane_agg(angles):
    x, y = mean_vec_from_dirs(angles.values) # component-wise mean
    mean = wind_dir_from_vec(x, y)
    diff = angular_difference(angles.values, mean)
    
    var = np.sum(np.square(diff)) / len(angles) # only 600 values so shouldn't have floating point problems
    min_ = (np.min(diff) + mean) % 360
    max_ = (np.max(diff) + mean) % 360
    
    out = np.array([mean, var, min_, max_]).reshape(1,-1)
    return pd.DataFrame(out)    

In [63]:
vane_aggs = []
for vane in vanes:
    agg = onehz[[vane]].resample('10min').agg(vane_agg)
    cols = pd.MultiIndex.from_tuples([(vane, calc) for calc in ['mean', 'var', 'min', 'max']])
    agg.columns = cols
    agg.index = agg.index.droplevel(1) # spurious zeros from component df indices
    vane_aggs.append(agg)

/mnt/c/Code/wind/turbulence_spectra/src/data/aggregate.py:131: RuntimeWarning: invalid value encountered in double_scalars
  angle = np.round(angle, 12) % 360  # convert interval from (-180, 180] to [0, 360)
/mnt/c/Code/wind/turbulence_spectra/src/features/build_features.py:20: RuntimeWarning: invalid value encountered in remainder
  return ((series1.squeeze().to_numpy() - series2.squeeze().to_numpy()) - 180) % -360 + 180
<ipython-input-56-88ae63e52d60>:11: RuntimeWarning: invalid value encountered in double_scalars
  min_ = (np.min(diff) + mean) % 360
<ipython-input-56-88ae63e52d60>:12: RuntimeWarning: invalid value encountered in double_scalars
  max_ = (np.max(diff) + mean) % 360


In [64]:
aggs = pd.concat(vane_aggs + [aggs, onehz[vanes].resample('10min').agg(['count'])], axis=1).sort_index(axis=1)

In [72]:
aggs.head(2).T

timestamp             2018-12-31 17:00:00  2018-12-31 17:10:00
Air_Temp_87m   count           600.000000           600.000000
               max             -15.477113           -15.577877
               mean            -15.691004           -15.798286
               min             -15.805410           -15.977685
               var               0.002746             0.004473
Cup_WS_C1_105m count           600.000000           600.000000
               max               8.917205             8.775206
               mean              7.490526             7.050964
               min               5.818164             5.438223
               var               0.288631             0.429616
Cup_WS_C1_130m count           600.000000           600.000000
               max               9.123878             8.726190
               mean              7.629745             7.180541
               min               6.183786             5.812447
               var               0.257707             0.359607
DeltaT_122_87m count           600.000000           600.000000
               max              -0.274433            -0.227949
               mean             -0.304441            -0.286377
               min              -0.330359            -0.339802
               var               0.000068             0.000622
Vane_WD_122m   count           600.000000           600.000000
               max              33.494219            31.750308
               mean             22.310701            18.815509
               min              11.305162             3.709933
               var              15.631823            23.242969
Vane_WD_87m    count           600.000000           600.000000
               max              36.993735            34.089358
               mean             23.033820            21.586551
               min              10.439399             3.040149
               var              17.199339            25.515584

In [73]:
gust = onehz[anemometers].rolling(window='3s').mean().resample('10min').agg(['max'])
gust.head(2)

,Cup_WS_C1_130m,Cup_WS_C1_105m
,max,max
timestamp,,
2018-12-31 17:00:00,8.685490,8.805074
2018-12-31 17:10:00,8.420425,8.546263


In [74]:
gust.columns = pd.MultiIndex.from_product([anemometers, ['3s_gust']])
gust.head(2)

,Cup_WS_C1_130m,Cup_WS_C1_105m
,3s_gust,3s_gust
timestamp,,
2018-12-31 17:00:00,8.685490,8.805074
2018-12-31 17:10:00,8.420425,8.546263


# Target
I will try to model high frequency energy content, defined as the power spectral density (PSD) integrated from 1/30 hz and up.

scipy FFT routines can't handle NaN values, so I'll linearly interpolate missing values. Linear interpolation preserves low frequency content but omits high frequencies. Later on, I will mitigate this bias by excluding aggregates with too many missing values.

In [75]:
from src.data.aggregate import integrated_spectral_densities

In [76]:
def agg_func(series):
    """Thin wrapper to convert pd.Series to ndarray."""
    return integrated_spectral_densities(series.to_numpy(),
                                         eval_freqs=(1/30,),
                                         square=True,
                                         sample_freq=1.
                                        )[0]

In [77]:
targets = onehz[anemometers].interpolate(method='linear').resample('10min').agg(agg_func)
targets.columns = pd.MultiIndex.from_product([anemometers, ['integrated_low_freq_ke']])
targets.head(2)

,Cup_WS_C1_130m,Cup_WS_C1_105m
,integrated_low_freq_ke,integrated_low_freq_ke
timestamp,,
2018-12-31 17:00:00,0.195348,0.212934
2018-12-31 17:10:00,0.306209,0.368811


In [78]:
aggs = pd.concat([aggs, gust, targets], axis=1).sort_index(axis=1)

In [79]:
aggs.describe()

Air_Temp_87m                                                          \
              count           max          mean           min           var   
count  52560.000000  51092.000000  51092.000000  51092.000000  51092.000000   
mean     583.238166      9.348125      8.970660      8.576753      0.052726   
std       98.866452     10.313859     10.278975     10.207363      0.280560   
min        0.000000    -20.911898    -21.085619    -21.236944      0.000000   
25%      600.000000      0.706636      0.324353      0.000000      0.005341   
50%      600.000000      9.102959      8.672078      8.243208      0.010997   
75%      600.000000     17.618618     17.209442     16.813549      0.027751   
max      600.000000     33.151836     32.608894     32.296959     14.963243   

      Cup_WS_C1_105m                                                     \
             3s_gust         count integrated_low_freq_ke           max   
count   49346.000000  52560.000000           5.256000e+04  49344.000000   
mean        6.725983    559.649715           8.848993e-01      6.929149   
std         4.949031    145.236654           1.810842e+00      5.150135   
min         0.279138      0.000000           3.044860e-10      0.279138   
25%         3.483986    600.000000           9.660502e-02      3.562775   
50%         5.335279    600.000000           2.869729e-01      5.476393   
75%         8.246281    600.000000           8.043675e-01      8.500767   
max        41.877602    600.000000           3.526694e+01     44.628307   

                                                Cup_WS_C1_130m                \
               mean           min           var        3s_gust         count   
count  49344.000000  49344.000000  49343.000000   49350.000000  52560.000000   
mean       4.701114      2.795991      1.124635       6.851781    559.642218   
std        3.594679      2.460562      2.212647       5.042704    145.224350   
min        0.262881      0.236069      0.000037       0.291123      0.000000   
25%        2.288911      1.009060      0.145414       3.512047    600.000000   
50%        3.695616      2.148362      0.380721       5.434299    600.000000   
75%        5.982199      3.844631      1.020492       8.483881    600.000000   
max       30.684881     22.873808     38.302898      40.572201    600.000000   

                                                                        \
      integrated_low_freq_ke           max          mean           min   
count           5.256000e+04  49348.000000  49348.000000  49348.000000   
mean            8.956128e-01      7.061840      4.836463      2.912085   
std             1.825647e+00      5.252625      3.712926      2.600091   
min             9.109066e-08      0.291123      0.281578      0.248087   
25%             9.206218e-02      3.591566      2.303326      0.975611   
50%             2.867365e-01      5.573894      3.794278      2.206494   
75%             8.258088e-01      8.732765      6.236612      4.092345   
max             3.499598e+01     41.961575     30.915714     24.476288   

                    DeltaT_122_87m                                            \
                var          count           max          mean           min   
count  49347.000000   52560.000000  51092.000000  51092.000000  51092.000000   
mean       1.136216     583.238166      0.012432     -0.176353     -0.348443   
std        2.227097      98.866452      0.587706      0.421301      0.332510   
min        0.000000       0.000000     -6.596122     -6.902417     -7.123314   
25%        0.137578     600.000000     -0.254551     -0.347299     -0.463276   
50%        0.380151     600.000000     -0.144750     -0.250621     -0.351264   
75%        1.047873     600.000000      0.106157     -0.097799     -0.256005   
max       38.216286     600.000000      9.393590      5.863889      5.342910   

                     Vane_WD_122m                                            \
                var         count          

In [80]:
aggs.to_parquet(data_dir / 'cup_10_min_aggs.parquet')